In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.master('local').appName("practice").getOrCreate()

In [5]:
spark

In [27]:
original_data = spark.read.csv(r'C:\Users\Kshitij kadu\Desktop\sample_data\Records.csv', header=True, inferSchema=True).\
                printSchema()

root
 |-- Emp ID: integer (nullable = true)
 |-- Name Prefix: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Middle Initial: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- E Mail: string (nullable = true)
 |-- Father's Name: string (nullable = true)
 |-- Mother's Name: string (nullable = true)
 |-- Mother's Maiden Name: string (nullable = true)
 |-- Date of Birth: string (nullable = true)
 |-- Time of Birth: string (nullable = true)
 |-- Age in Yrs.: double (nullable = true)
 |-- Weight in Kgs.: integer (nullable = true)
 |-- Date of Joining: string (nullable = true)
 |-- Quarter of Joining: string (nullable = true)
 |-- Half of Joining: string (nullable = true)
 |-- Year of Joining: integer (nullable = true)
 |-- Month of Joining: integer (nullable = true)
 |-- Month Name of Joining: string (nullable = true)
 |-- Short Month: string (nullable = true)
 |-- Day of Joining: integer (nullable = true)
 |

In [28]:
emp_df = spark.read.csv(r'C:\Users\Kshitij kadu\Desktop\sample_data\Records.csv', header=True, inferSchema=True).\
         select("Emp ID", "First Name", "Last Name", "Gender", "Date of Birth", "Salary","County","Year of Joining")

In [29]:
emp_df.show()

+------+----------+---------+------+-------------+------+------------+---------------+
|Emp ID|First Name|Last Name|Gender|Date of Birth|Salary|      County|Year of Joining|
+------+----------+---------+------+-------------+------+------------+---------------+
|198429|  Serafina|Bumgarner|     F|    9/21/1982| 69294|  Chautauqua|           2008|
|178566|  Juliette|     Rojo|     F|     5/8/1967|193912|  Montgomery|           2011|
|647173|     Milan| Krawczyk|     M|     4/4/1980|123681|Anne Arundel|           2012|
|847634|     Elmer|    Jason|     M|     4/9/1996| 93504|  Washington|           2017|
|260736|     Zelda|   Forest|     F|   11/27/1959|176642| Schenectady|           2014|
|811306|     Rhett|      Wan|     M|    7/14/1976| 59406|      Fresno|           2009|
|956633|       Hal|   Farrow|     M|    3/15/1967|164580|  Stanislaus|           1991|
|629539|       Del|Fernandez|     M|    8/13/1991|138662|     Portage|           2016|
|784160|     Corey|  Jackman|     M|    4/1

In [30]:
emp_df.printSchema()

root
 |-- Emp ID: integer (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Date of Birth: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- County: string (nullable = true)
 |-- Year of Joining: integer (nullable = true)



In [31]:
from pyspark.sql.functions import col, lit
from pyspark.sql.functions import rank, dense_rank, row_number

from pyspark.sql.window import Window

**get hightest paid emp in each country**

In [32]:
spec = Window.partitionBy("county").orderBy(col("Salary").desc())

In [33]:
emp_df.select("*").\
      withColumn("drank", dense_rank().over(spec)).\
      filter("drank=1").orderBy(col("Salary").desc()).show()

+------+----------+-----------+------+-------------+------+------------+---------------+-----+
|Emp ID|First Name|  Last Name|Gender|Date of Birth|Salary|      County|Year of Joining|drank|
+------+----------+-----------+------+-------------+------+------------+---------------+-----+
|583316|     Kenna|   Rothrock|     F|     2/6/1959|199971|     Madison|           1985|    1|
|506864|      Dick|        Sun|     M|    8/12/1959|199969|       Adair|           1987|    1|
|621655|       Sol| Pellegrino|     M|     6/8/1986|199958|    Accomack|           2013|    1|
|672252|       Jim|      Terry|     M|    1/18/1987|199954|Pottawatomie|           2010|    1|
|916763|  Floretta|     Macias|     F|    5/26/1973|199952|      Beaver|           1994|    1|
|404337|     Hosea|  Gustafson|     M|    11/4/1970|199952| Los Angeles|           2005|    1|
|740463|    Lanita|      Myatt|     F|    3/18/1986|199907|  Otter Tail|           2016|    1|
|566388|    Lennie|Baumgartner|     F|    6/30/196

In [63]:
emp_df.groupBy("County").sum("Salary").withColumn("sal", col("sum(Salary)")).show()

+--------------------+-----------+-------+
|              County|sum(Salary)|    sal|
+--------------------+-----------+-------+
|               Bucks|    1798995|1798995|
|              Grimes|     133943| 133943|
|           Worcester|    2605581|2605581|
|            Thurston|     303027| 303027|
|Skagway-Hoonah-An...|     168821| 168821|
|      East Feliciana|      77318|  77318|
|                Utah|     286249| 286249|
|               Tyler|     105695| 105695|
|             Hanover|     258744| 258744|
|          Charleston|    1094408|1094408|
|             Shannon|     365807| 365807|
|             Whigham|      85000|  85000|
|            Montcalm|     725627| 725627|
|                 Bow|     137317| 137317|
|               Pasco|     805124| 805124|
|            Angelina|     506885| 506885|
|              Hawaii|     760690| 760690|
|               Izard|     204622| 204622|
|          Deer Lodge|     157902| 157902|
|            Harrison|    1829477|1829477|
+----------

In [64]:
spec = Window.orderBy(col("Salary").desc())

In [70]:
emp_df.select("*").\
      withColumn("drank", dense_rank().over(spec)).\
      orderBy(col("Salary").desc()).show()

+------+----------+-----------+------+-------------+------+------------+---------------+-----+
|Emp ID|First Name|  Last Name|Gender|Date of Birth|Salary|      County|Year of Joining|drank|
+------+----------+-----------+------+-------------+------+------------+---------------+-----+
|583316|     Kenna|   Rothrock|     F|     2/6/1959|199971|     Madison|           1985|    1|
|506864|      Dick|        Sun|     M|    8/12/1959|199969|       Adair|           1987|    2|
|621655|       Sol| Pellegrino|     M|     6/8/1986|199958|    Accomack|           2013|    3|
|672252|       Jim|      Terry|     M|    1/18/1987|199954|Pottawatomie|           2010|    4|
|404337|     Hosea|  Gustafson|     M|    11/4/1970|199952| Los Angeles|           2005|    5|
|916763|  Floretta|     Macias|     F|    5/26/1973|199952|      Beaver|           1994|    5|
|740463|    Lanita|      Myatt|     F|    3/18/1986|199907|  Otter Tail|           2016|    6|
|854363|     Caleb|     Aiello|     M|     8/9/196